In [1]:
#@title Instalacje bibliotek
!pip install diffusers
!pip install transformers
!pip install omegaconf
!pip install tensorrt
!pip install accelerate
!pip install torch
!pip install gdown
!pip install safetensors
!pip install compel
!pip install xformers

  Using cached tensorrt-10.5.0-py2.py3-none-any.whl
  Using cached tensorrt-cu12-10.5.0.tar.gz (18 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.5.0-py2.py3-none-any.whl size=17556 sha256=d799f4aa6788aa67ebbd59c51eb2d37680caced5954ecf3177826b9e7da6b586
  Stored in directory: /root/.cache/pip/wheels/31/83/8a/84edbd0d600c1b334a5ac98e18626a458dc8a70d83d9c5ccbe
Successfully built tensorrt-cu12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 83.0 MB/s eta 0:00:00


In [2]:
#@title Mount Drive

from ipywidgets import (
    Textarea, Button, Dropdown, IntSlider, IntText, Image as IPythonImage, VBox, HBox, Tab, Output
)
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from google.colab import drive
from diffusers import StableDiffusionPipeline
from diffusers.pipelines.stable_diffusion import safety_checker
from diffusers.loaders import LoraLoaderMixin
from diffusers import schedulers
import torch
from safetensors.torch import load_file
import copy
import gdown
from PIL import Image, PngImagePlugin
import numpy as np
from os.path import isfile, join, exists
from os import listdir
import os
import subprocess
from datetime import datetime
from IPython.display import clear_output, update_display
import io
import sys
from compel import Compel
import warnings
import re
from collections import defaultdict

drive.mount("/content/gdrive/")

# Important variables
models_folder = "/content/StableDiffusionModels/"
loras_folder = "/content/lora_models/"
save_folder = "/content/gdrive/MyDrive/dnd_token_images"
colab_device = "cuda"
generator = torch.Generator(device = colab_device)
#samplers =
pipeline_copy = None
warnings.simplefilter('ignore', FutureWarning)

try:
  subprocess.run(["mkdir", save_folder], check=True)
  print(f"Directory '{save_folder}' created successfully.")
except subprocess.CalledProcessError as e:
  print(f"Failed to create directory '{save_folder}': {str(e)}")

# Turn off safety checker
def sc(self, clip_input, images) : return images, [False for i in images]
safety_checker.StableDiffusionSafetyChecker.forward = sc

# Until diffusers library won't fix their lora loading
def load_lora_weights(pipeline, checkpoint_path, multiplier, device, dtype, original_weights):

  restore_weights = bool(original_weights)

  # load base model
  LORA_PREFIX_UNET = "lora_unet"
  LORA_PREFIX_TEXT_ENCODER = "lora_te"
  # load LoRA weight from .safetensors
  state_dict = load_file(checkpoint_path, device)

  updates = defaultdict(dict)
  for key, value in state_dict.items():
      # it is suggested to print out the key, it usually will be something like below
      # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"

      layer, elem = key.split('.', 1)
      updates[layer][elem] = value

  index = 0
  # directly update weight in diffusers model
  for layer, elems in updates.items():
    index += 1

    if "text" in layer:
        layer_infos = layer.split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
        curr_layer = pipeline.text_encoder
    else:
        layer_infos = layer.split(LORA_PREFIX_UNET + "_")[-1].split("_")
        curr_layer = pipeline.unet

    # find the target layer
    temp_name = layer_infos.pop(0)
    while len(layer_infos) > -1:
        try:
            curr_layer = curr_layer.__getattr__(temp_name)
            if len(layer_infos) > 0:
                temp_name = layer_infos.pop(0)
            elif len(layer_infos) == 0:
                break
        except Exception:
            if len(temp_name) > 0:
                temp_name += "_" + layer_infos.pop(0)
            else:
                temp_name = layer_infos.pop(0)

    # get elements for this layer
    weight_up = elems['lora_up.weight'].to(dtype)
    weight_down = elems['lora_down.weight'].to(dtype)
    alpha = elems['alpha']
    if alpha:
        alpha = alpha.item() / weight_up.shape[1]
    else:
        alpha = 1.0

    if (restore_weights):
        curr_layer.weight.data = original_weights[index].clone().detach()
    else:
        original_weights[index] = curr_layer.weight.data.clone().detach()

        # update weight
        if len(weight_up.shape) == 4:
            curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up.squeeze(3).squeeze(2), weight_down.squeeze(3).squeeze(2)).unsqueeze(2).unsqueeze(3)
        else:
            curr_layer.weight.data += multiplier * alpha * torch.mm(weight_up, weight_down)

  return original_weights

LoraLoaderMixin.load_lora_weights = load_lora_weights

# Basic functions

def get_file_names(mypath):
  return [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

def get_loras_dict():
    lora_paths = get_file_names(loras_folder)
    loras_dict = {}
    for path in lora_paths:
        filename = os.path.basename(path)
        model_name = filename.split(".")[0]
        model_key = f"<lora:{model_name}:1>"
        loras_dict[model_key] = path
    return loras_dict

def download_file(url, folder):
  if url:
    command = ["wget", "--content-disposition", "-P", folder, url]
    print('Pobieranie...')
    return_code = subprocess.call(command)

    if return_code == 0:
        print('Pobrano plik.')
    else:
        print('Error. Nie pobrano pliku.')

def model_test(pipe):
  prompt = "a photograph of an astronaut riding a horse"
  return pipe(prompt).images[0]


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Mounted at /content/gdrive/
Directory '/content/gdrive/MyDrive/dnd_token_images' created successfully.


In [ ]:
#@title Pobieranie modeli dakkini
# Download custom lora models
gdown.download_folder("https://drive.google.com/drive/folders/1KGEH8seBJVHyOS1w5z1LXOB18nlwvQ2H?usp=sharing", quiet=True)

In [4]:
#@title Pobieranie base modelu
#@markdown Obsługiwane tylko modele fp16
model_url = "https://civitai.com/api/download/models/95489" #@param {type:"string"}

download_file(model_url, models_folder)

Pobieranie...
Pobrano plik.


In [ ]:
#@title Pobieranie dodatkowego modelu
lora_url = "" #@param {type:"string"}

download_file(lora_url, loras_folder)

Pobieranie...
Pobrano plik.


In [5]:
#@title Główne funkcje
# Diffusers functions

def create_pipelines(chosen_model):
  global pipeline_txt2img#, pipeline_img2img, pipeline_inpaint, pipeline_controlnet, pipeline_upscale

  lock_important_widgets(True)
  refresh_btn.disabled = True

  if not chosen_model['new']:
    raise Exception("Nie wybrano modelu.")

  if 'pipeline_txt2img' not in globals():
    pipeline_txt2img = StableDiffusionPipeline.from_single_file(chosen_model['new'],
                                                                torch_dtype=torch.float16,
                                                                revision="fp16",
                                                                safety_checker = None,
                                                                requires_safety_checker = False
                                                                ).to(colab_device)
    pipeline_txt2img.enable_xformers_memory_efficient_attention()

  sampler_options = globals()[f"pipeline_{tab_titles[tab_controller.selected_index]}"].scheduler.compatibles
  sampler_options = sorted([str(s).split('.')[-1].rstrip("'>") for s in sampler_options])
  sampler_dropdown.options = sampler_options

  lock_important_widgets(False)
  refresh_btn.disabled = False

def lock_important_widgets(lock):
  widgets_to_disable = [model_selector, script_widget, generate_btn_txt2img, sampler_dropdown, loras_dropdown]
  [setattr(w, "disabled", lock) for w in widgets_to_disable]

def refresh(_):
  global pipeline_txt2img
  lock_important_widgets(True)
  model_selector.disabled = False
  del pipeline_txt2img
  torch.cuda.empty_cache()

def output_widget_print(some_string):
  output_widget.clear_output()
  with output_widget:
    print(some_string)

def update_sampler(selected_sampler):
  pipe = globals()[f"pipeline_{tab_titles[tab_controller.selected_index]}"]

  scheduler_class = getattr(schedulers, selected_sampler['new'])

  # nie chce mi sie juz przegladac ich kodu, dopoki nie poprawia samplerow, robie to w ten sposob
  try:
      pipe.scheduler = scheduler_class.from_config(pipe.scheduler.config, use_karras_sigmas = True)
  except:
      pipe.scheduler = scheduler_class.from_config(pipe.scheduler.config)

  output_widget_print(f"Sampler zmieniony na: {selected_sampler['new']}.")

def loras_dropdown_function(selected_option):
  if selected_option['new']:
    prompt_widget.value += str(selected_option['new'])

def get_txt2img_values(_):
  values = {
      'prompt': prompt_widget.value,
      'neg_prompt': neg_prompt_widget.value,
      'sampling_steps': sampling_steps_widget.value,
      'pipeline': pipeline_txt2img,
      'width': image_width_widget.value,
      'height': image_height_widget.value,
      'img_nr': image_nr_widget.value,
      'guidance_scale': gs_widget.value,
      'seed': seed_widget.value,
      #'script': script_widget.value,
  }

  output_widget.clear_output()
  lock_important_widgets(True)

  with output_widget:
      byte_stream = io.BytesIO()
      if script_widget.value:
        globals().get(script_widget.value)(**values)
      else:
        create_images(**values).save(byte_stream, format='PNG')
      image_display.value = byte_stream.getvalue()

  lock_important_widgets(False)

def debug_xy(**values):
  lora_strength = np.arange(.1,1.1,.1)
  imgs = []

  dtype = torch.float16

  prompt, lora_dict, loras_prompt  = extract_loras_from_prompt(values['prompt'])

  percent_done = 0.0
  for lora in loras_prompt:
    for strength in lora_strength:

      # Add lora weights
      weight = values['pipeline'].load_lora_weights(checkpoint_path = lora_dict[lora.split(':')[1]],
                                                        multiplier = strength,
                                                        device = colab_device,
                                                        dtype = dtype,
                                                        original_weights = {})

      latents, seeds = noise_generator(1, values['seed'], values["pipeline"], values['height'], values['width'])
      images = image_generator(values['pipeline'],
                               prompt,
                               values['neg_prompt'],
                               values['height'],
                               values['width'],
                               values['sampling_steps'],
                               values['guidance_scale'],
                               values['img_nr'],
                               latents) # generate 1 image with current weights

      # Reset the weights
      values['pipeline'].load_lora_weights(
          checkpoint_path = lora_dict[lora.split(':')[1]],
          multiplier = strength,
          device = colab_device,
          dtype = dtype,
          original_weights = weight
      )

      del weight

      imgs.append(images[0])
      output_widget.clear_output(wait=True)
      percent_done += 100/(len(loras_prompt)*len(lora_strength))
      output_widget_print(f"{percent_done:.2f}%")

  if 'pipeline' in values:
    del values['pipeline']

  #save_image(imgs, dim1, dim2, loras, seeds, **values)

  save_image(imgs, len(loras_prompt), len(lora_strength), loras_prompt, seeds, **values)

  output_widget_print("Grid zapisany na dysku.")

def character_xy(**values):
  lora_strength = np.arange(.1,1.1,.1)
  imgs = []

  dtype = torch.float16

  prompt, lora_dict, loras_prompt  = extract_loras_from_prompt(values['prompt'])

  percent_done = 0.0
  for strength1 in lora_strength:
    for strength2 in lora_strength:

      # Add lora weights
      weight1 = values['pipeline'].load_lora_weights(checkpoint_path = lora_dict[loras_prompt[0].split(':')[1]],
                                                        multiplier = strength1,
                                                        device = colab_device,
                                                        dtype = dtype,
                                                        original_weights = {})

      weight2 = values['pipeline'].load_lora_weights(checkpoint_path = lora_dict[loras_prompt[1].split(':')[1]],
                                                        multiplier = strength2,
                                                        device = colab_device,
                                                        dtype = dtype,
                                                        original_weights = {})

      latents, seeds = noise_generator(1, values['seed'], values["pipeline"], values['height'], values['width'])
      images = image_generator(values['pipeline'],
                               prompt,
                               values['neg_prompt'],
                               values['height'],
                               values['width'],
                               values['sampling_steps'],
                               values['guidance_scale'],
                               values['img_nr'],
                               latents) # generate 1 image with current weights

      # Reset the weights
      values['pipeline'].load_lora_weights(
          checkpoint_path = lora_dict[loras_prompt[1].split(':')[1]],
          multiplier = strength2,
          device = colab_device,
          dtype = dtype,
          original_weights = weight2
      )

      values['pipeline'].load_lora_weights(
          checkpoint_path = lora_dict[loras_prompt[0].split(':')[1]],
          multiplier = strength1,
          device = colab_device,
          dtype = dtype,
          original_weights = weight1
      )

      del weight1
      del weight2

      imgs.append(images[0])
      output_widget.clear_output(wait=True)
      percent_done += 100/(len(lora_strength)*len(lora_strength))
      output_widget_print(f"{percent_done:.2f}%")

  if 'pipeline' in values:
    del values['pipeline']

  #save_image(imgs, dim1, dim2, loras, seeds, **values)

  save_image(imgs, len(lora_strength), len(lora_strength), loras_prompt, seeds, **values)

  output_widget_print("Grid zapisany na dysku.")

def extract_loras_from_prompt(prompt):
  loras_from_prompt_list = re.findall(r'(<[^>]+>)', prompt)
  loras_with_path_dict = {key.split(':')[1]: value for key, value in get_loras_dict().items()}
  prompt = re.sub(r'<[^>]+>', '', prompt)
  return prompt, loras_with_path_dict, loras_from_prompt_list

def noise_generator(num_images, custom_seed, pipe, height, width):
  latents = None
  seeds = []

  for _ in range(num_images):

    if custom_seed == -1:
        seed = generator.seed()
    else:
        seed = custom_seed

    seeds.append(seed)
    #generator = generator.manual_seed(seed)

    image_latents = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator.manual_seed(seed),
        device = colab_device,
        dtype = torch.float16
    )
    latents = image_latents if latents is None else torch.cat((latents, image_latents))

  return latents, seeds

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

def image_generator(pipe, prompt, neg_prompt, h, w, steps, gs, nr, latents):

  with torch.no_grad():

    compel_proc = Compel(tokenizer = pipe.tokenizer, text_encoder = pipe.text_encoder, truncate_long_prompts = False)
    conditioning = compel_proc.build_conditioning_tensor(prompt)
    negative_conditioning = compel_proc.build_conditioning_tensor(neg_prompt)
    [conditioning, negative_conditioning] = compel_proc.pad_conditioning_tensors_to_same_length([conditioning, negative_conditioning])

    images = pipe(prompt_embeds = conditioning,
                    negative_prompt_embeds = negative_conditioning,
                    height = h,
                    width = w,
                    num_inference_steps = steps,
                    guidance_scale = gs,
                    num_images_per_prompt = nr,
                    latents = latents).images
  return images

def create_images(**values):

  # Wyciagnij lory z prompta
  values['prompt'], lora_dict, loras_prompt_list  = extract_loras_from_prompt(values['prompt'])

  weights = []
  dtype = torch.float16

  for lora in loras_prompt_list:
    weights.append(values['pipeline'].load_lora_weights(checkpoint_path = lora_dict[lora.split(':')[1]],
                                                        multiplier = float(lora.split(':')[-1].strip('>')),
                                                        device = colab_device,
                                                        dtype = dtype,
                                                        original_weights = {}
                                                        ))

  latents, seeds = noise_generator(values['img_nr'], values['seed'], values["pipeline"], values['height'], values['width'])

  images = image_generator(values['pipeline'],
                           values['prompt'],
                           values["neg_prompt"],
                           values["height"],
                           values["width"],
                           values["sampling_steps"],
                           values['guidance_scale'],
                           values['img_nr'], latents)

  # Reset lora weights
  for i in range(len(loras_prompt_list)-1, -1, -1):
      values['pipeline'].load_lora_weights(
          checkpoint_path=lora_dict[loras_prompt_list[i].split(':')[1]],
          multiplier=float(loras_prompt_list[i].split(':')[-1].strip('>')),
          device=colab_device,
          dtype=dtype,
          original_weights=weights[i]
      )

  # brak tego oznacza memory leak :) debilu :)
  if 'pipeline' in values:
    del values['pipeline']

  # Na wszelki wypadek
  weights.clear()
  torch.cuda.empty_cache()

  img_dim = 2 if values['img_nr'] > 1 else 1

  return save_image(images, img_dim, img_dim, loras_prompt_list, seeds, **values)

def save_image(imgs, dim1, dim2, loras, seeds, **values):
  image = image_grid(imgs, dim1, dim2)
  metadata = PngImagePlugin.PngInfo()
  parameters = {"Prompt":values["prompt"],
                                "Negative prompt":values["neg_prompt"],
                                "Sampling steps":values["sampling_steps"],
                                "Size": f'{values["height"]}x{values["width"]}',
                                "Guidance Scale":values['guidance_scale'],
                                "Model":model_selector.value,
                                "Loras":loras,
                                "Sampler":sampler_dropdown.value,
                                "Seeds":seeds
                }

  metadata.add_text("Prompt", str(parameters))

  image.save(f"{save_folder}/img_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.png",pnginfo = metadata)

  image = image.resize((512, 512), Image.NEAREST)
  return image

In [7]:
#@title GUI
#@markdown Jeśli coś się zepsuje to kliknij R, odśwież komórkę, ewentualnie cały notatnik. Nie wszystkie samplery działają (problem używanej biblioteki), polecane samplery: DDIMScheduler, DPMSolverMultistepScheduler, KDPM2DiscreteScheduler (wolniejszy).
#@markdown Wielkość obrazków jest zablokowana na 512x512.
model_selector = Dropdown(
    options=get_file_names(models_folder), value=None, description='Model:',
    layout={'width': '750px', 'height': '40px', 'margin': '5px'}
)

refresh_btn = Button(
    description='R', button_style='success', icon='check', layout={'margin': '5px', 'width': '40px', 'height': '40px'}
)

selecting_section = HBox([model_selector, refresh_btn])

model_selector.observe(create_pipelines, names='value')

prompt_widget = Textarea(
    placeholder='Prompt', layout={'margin': '5px', 'width': 'auto', 'height': '90px'}
)
neg_prompt_widget = Textarea(
    placeholder='Negative prompt', layout={'margin': '5px', 'width': 'auto', 'height': '90px'}
)
generate_btn_txt2img = Button(
    description='Generate', button_style='success', disabled = True, icon='check', layout={'margin': '5px', 'width': '300px', 'height': '80px'}
)

loras_dropdown = Dropdown(
    options=[None]+list(get_loras_dict().keys()), value=None, description='Lora:',
    layout={'width': '240px', 'height': '40px', 'margin': '5px'}
)

loras_dropdown.observe(loras_dropdown_function, names = "value")

output_widget = Output(layout={'width': '400px', 'height': '80px', 'margin': '5px'})
output_box = VBox([output_widget])

sampling_steps_widget = IntSlider(
    value=50, description='Sampling Steps', max=100, min=1, layout={'width': '95%', 'margin': '5px'}
)
sampler_dropdown = Dropdown(
    options=[], value=None, disabled=True, description='Sampler/Scheduler',
    layout={'width': '500px', 'height': '40px', 'margin': '5px'}
)

sampler_dropdown.observe(update_sampler, names='value')

image_width_widget = IntSlider(
    value=512, description='Width', max=2048, min=128,step=8, layout={'width': '95%', 'margin': '5px'}
)
image_height_widget = IntSlider(
    value=512, description='Height', max=2048, min=128,step=8,layout={'width': '95%', 'margin': '5px'}
)
image_width_widget.disabled = True
image_height_widget.disabled = True

image_nr_widget = IntSlider(
    value=1, description='Number of Images to generate', max=4, min=1, step=3, layout={'width': '95%', 'margin': '5px'}
)
gs_widget = IntSlider(
    value=7, description='Guidance Scale', max=12, min=1, layout={'width': '95%', 'margin': '5px'}
)
seed_widget = IntText(value=-1, description='Seed', layout={'width': '300px', 'height': '30px', 'margin': '5px'})

script_widget = Dropdown(
    options=[None, 'debug_xy', 'character_xy'], value=None, description='Script',
    layout={'width': '260px', 'height': '30px', 'margin': '5px'}
)

image_display = IPythonImage(format='png', width=350, height=350)

btn_style = {'width': '140px', 'height': '60px', 'margin': '5px'}
buttons = [
    Button(description='Send to img2img', button_style='', icon='check', layout=btn_style),
    Button(description='Send to inpaint', button_style='', icon='check', layout=btn_style),
    Button(description='Send to controlnet', button_style='', icon='check', layout=btn_style),
    Button(description='Send to upscale', button_style='', icon='check', layout=btn_style)
]

# Arrange widgets in sections
prompt_section = VBox([prompt_widget, neg_prompt_widget], layout={'width': '95%', 'height': 'auto', 'margin': '5px'})
generate_section = VBox([generate_btn_txt2img, loras_dropdown, output_box], layout={'width': '95%', 'height': 'auto', 'margin': '5px'})
first_section = HBox([prompt_section, generate_section])

second_section = VBox(
    [sampling_steps_widget, sampler_dropdown, image_width_widget, image_height_widget,
     image_nr_widget, gs_widget, seed_widget, script_widget],
    layout={'width': '95%', 'height': 'auto', 'margin': '5px'}
)

btn_section = HBox(buttons, layout={'width': '95%', 'height': 'auto', 'margin': '5px'})
third_section = VBox([image_display,
                      #btn_section
                      ], layout={'width': '95%', 'height': 'auto', 'margin': '5px'})

fourth_section = HBox([second_section, third_section], layout={'width': '95%', 'height': 'auto', 'margin': '5px'})

txt2img_tab = VBox([first_section, fourth_section], layout={'width': '95%', 'height': 'auto', 'margin': '5px'})

generate_btn_txt2img.on_click(get_txt2img_values)
refresh_btn.on_click(refresh)
# Tu zrob img2img
tab_layout = {'width': '95%', 'height': 'auto', 'margin': '5px'}

txt2img_tab = VBox([first_section, fourth_section], layout=tab_layout)
img2img_tab = VBox([], layout=tab_layout)
inpaint_tab = VBox([], layout=tab_layout)
controlnet_tab = VBox([], layout=tab_layout)
upscale_tab = VBox([], layout=tab_layout)
pnginfo_tab = VBox([], layout=tab_layout)

tab_titles = ['txt2img',
              # 'img2img',
              # 'inpaint',
              # 'controlnet',
              # 'upscale',
              'pnginfo']

tab_controller = Tab(children=[txt2img_tab,
                              #  img2img_tab,
                              #  inpaint_tab,
                              #  controlnet_tab,
                              #  upscale_tab,
                               pnginfo_tab])

for i, title in enumerate(tab_titles):
    tab_controller.set_title(i, title)

gui = VBox([selecting_section, tab_controller])

display(gui)


RevisionNotFoundError: 404 Client Error. (Request ID: Root=1-6701c190-1cc1b1f11d5ef4ed5e524c47;0bdb003e-0633-4221-8201-f4273b8ca8fa)

Revision Not Found for url: https://huggingface.co/api/models/Lykon/dreamshaper-8/revision/fp16.
Invalid rev id: fp16